## Завдання 1 за допомогою Plotly

In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import numpy as np
import plotly.graph_objects as go

In [8]:
# Ініціалізація додатку
app = dash.Dash(__name__)

# Функція генерації гармоніки
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, samples=1000):
    x = np.linspace(0, 2 * np.pi, samples)
    harmonic = amplitude * np.sin(frequency * x + phase)
    noise = np.random.normal(noise_mean, noise_covariance, samples)
    return x, harmonic + noise if show_noise else harmonic

# Початкові параметри
initial_params = {
    "amplitude": 1.0,
    "frequency": 1.0,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True
}

# Layout
app.layout = html.Div(
    style={'backgroundColor': '#000000', 'color': 'white', 'padding': '20px'},
    children=[
        html.H1("Інтерактивна побудова гармоніки з шумом", style={'color': 'white'}),
        dcc.Graph(id='harmonic-plot'),
        html.Label("Amplitude:", style={'color': 'white'}),
        dcc.Slider(id='amplitude-slider', min=0.1, max=5.0, step=0.1, value=initial_params["amplitude"]),
        html.Label("Frequency:", style={'color': 'white'}),
        dcc.Slider(id='frequency-slider', min=0.1, max=5.0, step=0.1, value=initial_params["frequency"]),
        html.Label("Phase:", style={'color': 'white'}),
        dcc.Slider(id='phase-slider', min=0, max=2 * np.pi, step=0.1, value=initial_params["phase"]),
        html.Label("Noise Mean:", style={'color': 'white'}),
        dcc.Slider(id='noise-mean-slider', min=-1, max=1, step=0.1, value=initial_params["noise_mean"]),
        html.Label("Noise Covariance:", style={'color': 'white'}),
        dcc.Slider(id='noise-covariance-slider', min=0.01, max=1, step=0.01, value=initial_params["noise_covariance"]),
        html.Label("Відображати шум:", style={'color': 'white'}),
        dcc.Checklist(
            id='show-noise-checkbox', 
            options=[{'label': 'Так', 'value': 'show_noise', 'style': {'color': 'white'}}],
            value=['show_noise']
        ),
        html.Button("Reset", id='reset-button', style={'color': 'white', 'backgroundColor': '#333333'}),
        html.Div(id='hidden-div', style={'display': 'none'})
    ]
)

# Callback для оновлення графіку
@app.callback(
    Output('harmonic-plot', 'figure'),
    [
        Input('amplitude-slider', 'value'),
        Input('frequency-slider', 'value'),
        Input('phase-slider', 'value'),
        Input('noise-mean-slider', 'value'),
        Input('noise-covariance-slider', 'value'),
        Input('show-noise-checkbox', 'value'),
    ]
)
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise):
    show_noise_flag = 'show_noise' in show_noise
    x, y = harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise_flag)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Гармоніка'))
    fig.update_layout(
        title="Гармоніка з шумом", 
        xaxis_title="Час", 
        yaxis_title="Амплітуда",
        paper_bgcolor="#000000",
        plot_bgcolor="#000000",
        font=dict(color="white")
    )
    return fig

# Callback для скидання параметрів
@app.callback(
    [Output('amplitude-slider', 'value'),
     Output('frequency-slider', 'value'),
     Output('phase-slider', 'value'),
     Output('noise-mean-slider', 'value'),
     Output('noise-covariance-slider', 'value'),
     Output('show-noise-checkbox', 'value')],
    [Input('reset-button', 'n_clicks')]
)
def reset_params(n_clicks):
    if n_clicks:
        return (initial_params["amplitude"], initial_params["frequency"], initial_params["phase"],
                initial_params["noise_mean"], initial_params["noise_covariance"], ['show_noise'])
    return dash.no_update

if __name__ == '__main__':
    app.run_server(debug=True)